# Previsão de renda

### 4 elementos importantes
- Esse notebook
- Streamlit com as análises
- Seu Github com o projeto
- Vídeo no readme do github mostrando o streamlit

## Etapa 1 CRISP - DM: Entendimento do negócio

<span><b>Principal objetivo:</b> A entrega de um modelo que seja capaz de prever com precisão máxima a renda de um indivíduo levando em consideração diversos fatores sobre o mesmo.</span>

<span><b>O recurso principal para realizar a previsão se trata de um Dataset com diversas variáveis explicativas.</b></span>

<span><b>Os principais interessados na realização da previsão de renda podem ser:</b> Seguradoras, Bancos e outras instituições financeiras.</span>


## Etapa 2 Crisp-DM: Entendimento dos dados
<span style="color:red">ESCREVER AQUI</span>


### Dicionário de dados

<span style="color:red">ESCREVER AQUI</span>


| Variável                | Descrição                                           | Tipo         |
| ----------------------- |:---------------------------------------------------:| ------------:|
| data_ref                |  ESCREVER AQUI                                      | datatime|
| id_cliente              |  ESCREVER AQUI                                      | int|
| sexo                    |  ESCREVER AQUI                                      | object|
| posse_de_veiculo        |  ESCREVER AQUI                                      | bool|
| posse_de_imovel         |  ESCREVER AQUI                                      | bool|
| qtd_filhos              |  ESCREVER AQUI                                      | int|
| tipo_renda              |  ESCREVER AQUI                                      | object|
| educacao                |  ESCREVER AQUI                                      | object|
| estado_civil            |  ESCREVER AQUI                                      | object|
| tipo_residencia         |  ESCREVER AQUI                                      | object|
| idade                   |  ESCREVER AQUI                                      | int|
| tempo_emprego           |  ESCREVER AQUI                                      | float|
| qt_pessoas_residencia   |  ESCREVER AQUI                                      | float|
| renda                   |  ESCREVER AQUI                                      | float|





#### Carregando os pacotes
É considerado uma boa prática carregar os pacotes que serão utilizados como a primeira coisa do programa.

<span style="color:red">ESCREVER AQUI</span>


#### Carregando os dados
O comando pd.read_csv é um comando da biblioteca pandas (pd.) e carrega os dados do arquivo csv indicado para um objeto *dataframe* do pandas.

<span style="color:red">ESCREVER AQUI</span>


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#from pandas_profiling import ProfileReport

In [3]:
renda = pd.read_csv('./input/previsao_de_renda.csv')

In [4]:
renda.head(1)

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.60274,1.0,8060.34


In [7]:
renda.dtypes

Unnamed: 0                 int64
data_ref                  object
id_cliente                 int64
sexo                      object
posse_de_veiculo            bool
posse_de_imovel             bool
qtd_filhos                 int64
tipo_renda                object
educacao                  object
estado_civil              object
tipo_residencia           object
idade                      int64
tempo_emprego            float64
qt_pessoas_residencia    float64
renda                    float64
dtype: object

#### Entendimento dos dados - Univariada
Nesta etapa tipicamente avaliamos a distribuição de todas as variáveis. 

In [ ]:
prof = ProfileReport(renda, explorative=True, minimal=True)
prof


In [5]:
prof.to_file('./output/renda_analisys.html')

<span style="color:red">ESCREVER AQUI</span>


### Entendimento dos dados - Bivariadas




<span style="color:red">ESCREVER AQUI</span>


## Etapa 3 Crisp-DM: Preparação dos dados
Nessa etapa realizamos tipicamente as seguintes operações com os dados:

 - **seleção**: Já temos os dados selecionados adequadamente?
 - **limpeza**: Precisaremos identificar e tratar dados faltantes
 - **construção**: construção de novas variáveis
 - **integração**: Temos apenas uma fonte de dados, não é necessário integração
 - **formatação**: Os dados já se encontram em formatos úteis?



<span style="color:red">ESCREVER AQUI</span>


## Etapa 4 Crisp-DM: Modelagem
Nessa etapa que realizaremos a construção do modelo. Os passos típicos são:
- Selecionar a técnica de modelagem
- Desenho do teste
- Avaliação do modelo


<span style="color:red">ESCREVER AQUI</span>


### Rodando o modelo


<span style="color:red">ESCREVER AQUI</span>


## Etapa 5 Crisp-DM: Avaliação dos resultados


<span style="color:red">ESCREVER AQUI</span>


## Etapa 6 Crisp-DM: Implantação
Nessa etapa colocamos em uso o modelo desenvolvido, normalmente implementando o modelo desenvolvido em um motor que toma as decisões com algum nível de automação.